# Assignment 2

Reading the text and using sentence tokenizer

In [172]:
import re
with open("C:/Users/Mohit Mina/Downloads/speeches.txt","r",encoding="utf-8") as file:
    text=file.read()
    text=text.lower()
    text=re.sub('\n','',text)
from nltk.tokenize import sent_tokenize
text = sent_tokenize(text)
print(len(text))


12601


In [173]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(text, test_size = 0.2)

print(len(train))
print(len(test))


10080
2521


In [174]:
import string
train_set=[]
for i in train:
    i = re.sub(r'[^\w\s]','',i)
    train_set.append(i)
print(len(train_set))
        

10080


In [175]:
import nltk
from nltk import word_tokenize
tokens=[]
for i in range(len(train_set)):
    tokens.append('<s>')
    tokens_of_sent=word_tokenize(train_set[i])
    for j in range(len(tokens_of_sent)):
        tokens.append(tokens_of_sent[j])
    tokens.append('</s>')


In [176]:
from collections import Counter
from nltk.util import ngrams
unigram=tokens


In [128]:
c_unigram=len(unigram)
bigrams = list(nltk.bigrams(tokens))
trigrams = list(nltk.trigrams(tokens))
quadgrams = list(ngrams(tokens,4))
print("number of possible unigrams = ",c_unigram)
print("actual number of unigrams =",c_unigram)

print("number of possible bigrams = ",c_unigram**2)
print("actual number of bigrams =",len(bigrams))

print("number of possible trigrams = ",c_unigram**3)
print("actual number of trigrams =",len(trigrams))

print("number of possible quadgrams = ",c_unigram**4)
print("actual number of quadgrams =",len(quadgrams))

number of possible unigrams =  149771
actual number of unigrams = 149771
number of possible bigrams =  22431352441
actual number of bigrams = 149770
number of possible trigrams =  3359566086441011
actual number of trigrams = 149769
number of possible quadgrams =  503165572332356658481
actual number of quadgrams = 149768


In [129]:
def MLE_unigrams(word):
    prob=unigram.count(word)/(c_unigram)
    return prob
print(MLE_unigrams('look'))

0.0019496431218326644


In [130]:
def MLE_bigrams(word):
    prob=bigrams.count(word)/(unigram.count(word[0]))
    return prob
print(MLE_bigrams(('be','elected')))

0.0010787486515641855


In [131]:
def MLE_trigrams(word):
    prob=trigrams.count(word)/(bigrams.count(word[:2]))
    return prob
print(MLE_trigrams(('shouldnt','have','done')))

0.1


In [132]:
def MLE_quadgrams(word):
    prob=quadgrams.count(word)/(trigrams.count(word[:3]))
    return prob
print(MLE_quadgrams(('way','its','going','to')))

0.6666666666666666


In [ ]:
import numpy as np

def bigrams_generator():
    list_=[]
    sentence="<s>"
    start='<s>'
    while(1):
        for i in bigrams:
            if i[0]==start:
                list_.append(i)
        prob_list=[MLE_bigrams(i) for i in list_]
        sum_of_prob=sum(prob_list)
        prob_list=[i/sum_of_prob for i in prob_list]          #prob(i)/sum of prob 
        ans = list(np.random.multinomial(1,prob_list))
        ind = np.argmax(ans)
        new_word = list_[ind][1]
        sentence=sentence+" "+new_word
        if (new_word=='</s>'):
            break
        else:
            start = new_word
    return sentence

In [171]:
def generate(k):
    if k==1:
        return bigrams_generator()
    elif k==2:
        return trigrams_generator()
    elif k==3:
        return quadgrams_generator()
    else:
        return "Model no. not valid"

In [177]:
import string
test_set=[]
for i in test:
    i = re.sub(r'[^\w\s]','',i)
    test_set.append(i)
print(len(test_set))
  

2521


In [178]:
import nltk
from nltk import word_tokenize
tokens=[]
for i in range(len(train_set)):
    tokens.append('<s>')
    tokens_of_sent=word_tokenize(train_set[i])
    for j in range(len(tokens_of_sent)):
        tokens.append(tokens_of_sent[j])
    tokens.append('</s>')


# Perplexity

In [168]:
import math
def log_prob(sentence,k):
    prob=0
    if k == 0:
        prob = 0
        for i in range(len(sentence)):
            try:
                prob += math.log(MLE_unigrams(sentence[i]))
            except:
                continue
        return prob
    elif k == 1:
        try:
            prob = math.log(MLE_unigrams(sentence[0]))
        except:
            pass
        for i in range(len(sentence)-1):
            try:
                prob += math.log(MLE_bigrams(sentence[i:i+2]))
            except:
                continue
        return prob
    
    elif k == 2:
        try:    
            prob = math.log(MLE_unigrams(sentence[0]))+math.log(MLE_bigrams(sentence[0:2]))
        except:
            pass
        for i in range(len(sentence)-2):
            try:
                prob += math.log(MLE_trigrams(sentence[i:i+3]))
            except:
                continue        
        return prob
    
    elif k == 3:
        try:    
            prob = math.log(MLE_unigrams(sentence[0]))+math.log(MLE_bigrams(sentence[0:2]))+math.log(MLE_trigrams(sentence[0:3]))
        except:
            pass
        for i in range(len(sentence)-3):
            try:
                prob += math.log(MLE_quadgrams(sentence[i:i+4]))
            except:
                continue
        return prob


In [170]:
perp = math.exp(-1*log_prob(test,3)/len(test))
print ("Perplexity : ",perp)